In [9]:
// Note use of IF NOT EXISTS syntax to avoid errors if already present
CREATE KEYSPACE IF NOT EXISTS my_keyspace WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1} ;

In [39]:
USE my_keyspace;

In [40]:
CREATE TABLE IF NOT EXISTS user ( first_name text, last_name text, title text, PRIMARY KEY (first_name));

In [41]:
// Write a couple of rows, but only if they don't exist already
INSERT INTO user (first_name, last_name, title) VALUES ('Bill', 'Nguyen', 'Mr.') IF NOT EXISTS;
INSERT INTO user (first_name, last_name) VALUES ('Mary', 'Rodriguez') IF NOT EXISTS;

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [42]:
//
// UUID examples
//

// Add a unique identifier using a uuid
ALTER TABLE user ADD id uuid;

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [43]:
// Allow Cassandra to auto-assign a uuid
UPDATE user SET id = uuid() WHERE first_name = 'Mary';

In [15]:
// View the id that was set
SELECT first_name, id FROM user WHERE first_name = 'Mary';

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
 first_name | id
------------+--------------------------------------
       Mary | 93e602db-00c9-4af2-8900-0c09e828725c

(1 rows)

In [16]:
//
// Set examples
//

// Add a set to contain email addresses
ALTER TABLE user ADD emails set<text>;

In [17]:
// Add an email address and check that it was added successfully
UPDATE user SET emails = { 'mary@example.com' } WHERE first_name = 'Mary';
SELECT emails FROM user WHERE first_name = 'Mary';

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
 emails
----------------------
 {'mary@example.com'}

(1 rows)

In [18]:
// Add another email address using concatenation
UPDATE user SET emails = emails + {'mary.mcdonald.AZ@gmail.com' } WHERE first_name = 'Mary';
SELECT emails FROM user WHERE first_name = 'Mary';

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
 emails
----------------------------------------------------
 {'mary.mcdonald.AZ

In [19]:
//
// List examples
//

// Modify the user table to add a list of phone numbers
ALTER TABLE user ADD phone_numbers list<text>;

In [20]:
// Add a phone number for Mary and check that it was added successfully
UPDATE user SET phone_numbers = [ '1-800-999-9999' ] WHERE first_name = 'Mary';
SELECT phone_numbers FROM user WHERE first_name = 'Mary';

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [21]:
// Add a second number by appending it:
UPDATE user SET phone_numbers = phone_numbers + [ '480-111-1111' ] WHERE first_name = 'Mary';
SELECT phone_numbers FROM user WHERE first_name = 'Mary';

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [22]:
// Replace an individual item in the list referenced by its index
UPDATE user SET phone_numbers[1] = '480-111-1111' WHERE first_name = 'Mary';

In [23]:
// Use the subtraction operator to remove a list item matching a specified value
UPDATE user SET phone_numbers = phone_numbers - [ '480-111-1111' ] WHERE first_name = 'Mary';

In [24]:
// Delete a specific item directly using its index
DELETE phone_numbers[0] from user WHERE first_name = 'Mary';

In [25]:
//
// Map examples
//

// Add a map attribute to store information about user logins (timed in seconds) keyed by a timestamp (timeuuid)
ALTER TABLE user ADD login_sessions map<timeuuid, int>;

In [26]:
// Add a couple of login sessions for Mary and see the results
// Use the now() function to allow Cassandra to set the timestamp
UPDATE user SET login_sessions = { now(): 13, now(): 18} WHERE first_name = 'Mary';

In [27]:
SELECT login_sessions FROM user WHERE first_name = 'Mary';

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [28]:
//
// User Defined Type (UDT) examples
//

// Create a UDT for address information
CREATE TYPE address (street text, city text, state text, zip_code int);

In [29]:
// Can we use this UDT in a map?
ALTER TABLE user ADD addresses map<text, address>;

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [30]:
// Freeze the UDT so we can use it in a map
// freezing means we cannot access individual fields of the UDT but must select or insert the entire object at once
ALTER TABLE user ADD addresses map<text, frozen<address>>;

In [31]:
// Add a home address for Mary
UPDATE user SET addresses = addresses +  {'home': { street: '7712 E. Broadway', city: 'Tucson',
  state: 'AZ', zip_code: 85715} } WHERE first_name = 'Mary';

In [32]:
//
// Index examples
//

// Query based on a non-primary key column
// Why doesn't this work?
SELECT * FROM user WHERE last_name = 'Nguyen';

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [33]:
// Create a secondary index for the last_name column.
CREATE INDEX ON user ( last_name );

In [34]:
// Now try the query again
SELECT * FROM user WHERE last_name = 'Nguyen';

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [35]:
// View the output of the describe command to see the full index definition
// We didn't name the index, so Cassandra assigned a default name
DESCRIBE KEYSPACE;

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [36]:
// Create indexes on other attributes if desired, even collections
// Note that queries based on indexes are typically more expensive, as they involve talking to more nodes
CREATE INDEX ON user ( addresses );
CREATE INDEX ON user ( emails );
CREATE INDEX ON user ( phone_numbers );

In [37]:
// Drop indexes we no longer want maintained
DROP INDEX user_last_name_idx;
DROP INDEX user_addresses_idx;
DROP INDEX user_emails_idx;
DROP INDEX user_phone_numbers_idx;